In [24]:
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
import csv
import random
import math

def sig(x): #activation function: sigmoid
    return 1 /(1+(math.e**-x))
def mse(tar,out): #error check (means squared error)
    return ((tar-out)**2)/2
def grad(o,t,h): #gradient descent from output
    return (o-t)*o*(1-o)*h

dataset = "./datasets/iris.csv"
#desc: 4 input node, 3 output node, 5 hidden layer's node, 1 hidden layer, 20 first weights and 15 second weights
#1st output: Iris-setosa, 2nd output: Iris-versicolor, 3rd output: Iris-virginica

#initializations
a = 0.5 #learning rate
bh = 0.35 #bias for hidden layer
bo = 0.6 #bias for output layer
w1 = np.random.rand(5,4) #weights for input - hidden layer
w2 = np.random.rand(3,5) #weights for hidden layer - output
#w1 = w1[0]
#w2 = w2[0]
print(w1)
print(w2)

testd1 = [5.1,3.5,1.4,0.2] #test data, will be changed with csv
testd1o = [1,0,0] #expected output, will be changed with csv

#testd1.append(1)
#w1.append(bh)
neth = [0,0,0,0,0]
h = [0,0,0,0,0]
neto = [0,0,0]
o = [0,0,0]

#forward pass
for i,j in enumerate(neth):
    neth[i] = np.dot(testd1,w1[i])+bh
    h[i] = sig(neth[i])

for i,j in enumerate(neto):
    neto[i] = np.dot(h,w2[i])+bo
    o[i] = sig(neto[i])
    
#error check
etot = mse(testd1o[0],o[0])+mse(testd1o[1],o[1])+mse(testd1o[2],o[2])

#backward pass
#output
w2t = w2

for i,k in enumerate(w2): #for the output
    for j,k in enumerate(w2[i]): #for the weight
        w2[i][j] = w2t[i][j]-a*grad(o[i],testd1o[i],h[j])
        print(w2[i][j])
        
'''
w2[2][4] = w2t[2][4]-a*grad(o[2],testd1o[2],h[4])
w2[2][3] = w2t[2][3]-a*grad(o[2],testd1o[2],h[3])
w2[2][2] = w2t[2][2]-a*grad(o[2],testd1o[2],h[2])
w2[2][1] = w2t[2][1]-a*grad(o[2],testd1o[2],h[1])
w2[2][0] = w2t[2][0]-a*grad(o[2],testd1o[2],h[0])
w2[1][4] = w2t[1][4]-a*grad(o[1],testd1o[1],h[4])
w2[1][3] = w2t[1][3]-a*grad(o[1],testd1o[1],h[3])
w2[1][2] = w2t[1][2]-a*grad(o[1],testd1o[1],h[2])
w2[1][1] = w2t[1][1]-a*grad(o[1],testd1o[1],h[1])
w2[1][0] = w2t[1][0]-a*grad(o[1],testd1o[1],h[0])
w2[0][4] = w2t[0][4]-a*grad(o[0],testd1o[0],h[4])
w2[0][3] = w2t[0][3]-a*grad(o[0],testd1o[0],h[3])
w2[0][2] = w2t[0][2]-a*grad(o[0],testd1o[0],h[2])
w2[0][1] = w2t[0][1]-a*grad(o[0],testd1o[0],h[1])
w2[0][0] = w2t[0][0]-a*grad(o[0],testd1o[0],h[0])
'''
'''
#forward pass
#-for loop to number of first hidden neurons
neth[0] = np.dot(testd1,w1[:4])+bh
neth[1] = np.dot(testd1,w1[4:8])+bh
neth[2] = np.dot(testd1,w1[8:12])+bh
neth[3] = np.dot(testd1,w1[12:16])+bh
neth[4] = np.dot(testd1,w1[16:])+bh
#-for loop to number of first hidden neurons
h[0] = sig(neth[0])
h[1] = sig(neth[1])
h[2] = sig(neth[2])
h[3] = sig(neth[3])
h[4] = sig(neth[4])
#-for loop to number of output neurons
neto[0] = np.dot(h,w2[:5])+bo
neto[1] = np.dot(h,w2[5:10])+bo
neto[2] = np.dot(h,w2[10:])+bo
#-for loop to number of output neurons
o[0] = sig(neto[0])
o[1] = sig(neto[1])
o[2] = sig(neto[2])

#error check
etot = mse(testd1o[0],o[0])+mse(testd1o[1],o[1])+mse(testd1o[2],o[2])

#backward pass



print("h1: "+str(h1))
print("h2: "+str(h2))
print("h3: "+str(h3))
print("h4: "+str(h4))
print("h5: "+str(h5))
'''

[[0.89670799 0.77580055 0.54002661 0.8838205 ]
 [0.15650362 0.59993704 0.221231   0.60552696]
 [0.13036567 0.29654391 0.92686174 0.32741636]
 [0.26338614 0.46528207 0.53837756 0.36311103]
 [0.71430042 0.98952411 0.15444464 0.31506508]]
[[0.98054758 0.10846806 0.19487526 0.68942262 0.68395735]
 [0.34934595 0.27771594 0.03154701 0.74517821 0.08991087]
 [0.25541632 0.32935251 0.24331253 0.06597398 0.58641756]]
0.9812344994832087
0.10913818014439637
0.19554045773473056
0.690098980661899
0.6846440997847852
0.3053039528574699
0.23475064377166704
-0.011102371534780713
0.7018127959636342
0.04587977417961348
0.2110555370069923
0.2860762237341186
0.20035443943656006
0.02229468211937699
0.5420677592086046


'\n#forward pass\n#-for loop to number of first hidden neurons\nneth[0] = np.dot(testd1,w1[:4])+bh\nneth[1] = np.dot(testd1,w1[4:8])+bh\nneth[2] = np.dot(testd1,w1[8:12])+bh\nneth[3] = np.dot(testd1,w1[12:16])+bh\nneth[4] = np.dot(testd1,w1[16:])+bh\n#-for loop to number of first hidden neurons\nh[0] = sig(neth[0])\nh[1] = sig(neth[1])\nh[2] = sig(neth[2])\nh[3] = sig(neth[3])\nh[4] = sig(neth[4])\n#-for loop to number of output neurons\nneto[0] = np.dot(h,w2[:5])+bo\nneto[1] = np.dot(h,w2[5:10])+bo\nneto[2] = np.dot(h,w2[10:])+bo\n#-for loop to number of output neurons\no[0] = sig(neto[0])\no[1] = sig(neto[1])\no[2] = sig(neto[2])\n\n#error check\netot = mse(testd1o[0],o[0])+mse(testd1o[1],o[1])+mse(testd1o[2],o[2])\n\n#backward pass\n\n\n\nprint("h1: "+str(h1))\nprint("h2: "+str(h2))\nprint("h3: "+str(h3))\nprint("h4: "+str(h4))\nprint("h5: "+str(h5))\n'